In [26]:
import requests
import bcrypt
import pybase64
import time
import urllib.request
import urllib.parse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyperclip
import pyautogui
import traceback
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import sys
from selenium.webdriver.chrome.service import Service

def get_token(client_id, client_secret, type_="SELF") -> str:
    timestamp = str(int((time.time()-3) * 1000))
    pwd = f'{client_id}_{timestamp}'
    hashed = bcrypt.hashpw(pwd.encode('utf-8'), client_secret.encode('utf-8'))
    client_secret_sign = pybase64.standard_b64encode(hashed).decode('utf-8')

    headers = {"content-type": "application/x-www-form-urlencoded"}
    data_ = {
        "client_id": client_id,
        "timestamp": timestamp,
        "client_secret_sign": client_secret_sign,
        "grant_type": "client_credentials",
        "type": type_
    }

    query = urllib.parse.urlencode(data_)
    url = 'https://api.commerce.naver.com/external/v1/oauth2/token?' + query
    res = requests.post(url=url, headers=headers)
    res_data = res.json()

    while True:
        if 'access_token' in res_data:
            token = res_data['access_token']
            return token
        else:
            print(f'[{res_data}] 토큰 요청 실패')
            time.sleep(1)

token = get_token(client_id='7kL0hPwR48u8tNRzwFGZt6', client_secret='$2a$04$ZbDklubhU1hHokgaZvSCt.')
print(f'발급된 토큰 : ', token)

def get_new_order_list():
    from datetime import datetime, timedelta

    headers = {'Authorization': token}
    url = 'https://api.commerce.naver.com/external/v1/pay-order/seller/product-orders/last-changed-statuses'

    now = datetime.now()
    # before_date = now - timedelta(hours=3) #3시간전
    # before_date = now - timedelta(seconds=10) #10초전
    # before_date = now - timedelta(minutes=10) #10분전
    before_date = now - timedelta(hours=3)
    iosFormat = before_date.astimezone().isoformat()

    params = {
            'lastChangedFrom' : iosFormat, #조회시작일시
            'lastChangedType' : 'PAYED', #최종변경구분(PAYED : 결제완료, DISPATCHED : 발송처리)
        }

    res = requests.get(url=url, headers=headers, params=params)
    res_data = res.json()

    if 'data' not in res_data: #조회된 정보가 없을 경우 data키 없음
        print('주문 내역 없음')
        return False

    data_list = res_data['data']['lastChangeStatuses']

    for data in data_list:
        print(data) #주문 정보


get_new_order_list()

def get_order_detail():
    import datetime

    headers = {'Authorization': token}
    url = 'https://api.commerce.naver.com/external/v1/pay-order/seller/product-orders/query'

    params = {
        'productOrderIds' : ['']
    }

    res = requests.post(url=url, headers=headers, json=params)
    res_data = res.json()

    # print(res_data)
    if 'data' not in res_data:
        return False

    for data in res_data['data']:
        for d in data.keys():
            for d2 in data[d].keys():
                print(f'{d2} : {data[d][d2]}')

def get_new_order_list():
    from datetime import datetime, timedelta

    headers = {'Authorization': token}
    url = 'https://api.commerce.naver.com/external/v1/pay-order/seller/product-orders/last-changed-statuses'

    now = datetime.now()
    before_date = now - timedelta(hours=3) #3시간전
    iosFormat = before_date.astimezone().isoformat()

    params = {
            'lastChangedFrom' : iosFormat, #조회시작일시
            'lastChangedType' : 'PAYED', #최종변경구분(PAYED : 결제완료, DISPATCHED : 발송처리)
        }

    res = requests.get(url=url, headers=headers, params=params)
    res_data = res.json()

    if 'data' not in res_data: #조회된 정보가 없을 경우 data키 없음
        print('주문 내역 없음')
        return False

    data_list = res_data['data']['lastChangeStatuses']

    if data_list: # 주문 정보가 있다면
        print("주문이 발생했습니다. 이메일을 보냅니다.")
        send_email() # 이메일을 보내는 코드를 실행
    else:
        print("새로운 주문 정보가 없습니다.")


def send_email(): # 여기에 이메일을 보내는 코드를 작성합니다.
    # 네이버 로그인
    url = "https://nid.naver.com/nidlogin.login?mode=form&url=https://www.naver.com/"
    browser = webdriver.Chrome()
    browser.implicitly_wait(10) #페이지 로딩 기다려줌
    browser.maximize_window() # 화면 최대화
    browser.get(url) # 페이지 열기

    browser.find_element(By.CSS_SELECTOR,'#id') #예외처리에 필요 이 구문이 없으면 아이디가 클립보드에 계속 복사됨
    browser.implicitly_wait(5) #페이지 로딩 기다려줌
    nid='greenrabbit99'           #아이디 입력부분
    pyperclip.copy(nid)
    browser.find_element(By.CSS_SELECTOR,'#id').send_keys(Keys.CONTROL+'v')
    time.sleep(2)
    npw='rhdiddl321!'         #비밀번호 입력부분
    pyperclip.copy(npw)
    secure='blank'
    browser.find_element(By.CSS_SELECTOR,'#pw').send_keys(Keys.CONTROL + 'v')
    pyperclip.copy(secure)  #비밀번호 보안을위해 클립보드에 blank저장
    browser.find_element(By.XPATH,'//*[@id="log.login"]').click()

    #스마트스토어 로그인
    url = "https://accounts.commerce.naver.com/login?url=https%3A%2F%2Fsell.smartstore.naver.com%2F%23%2Flogin-callback"
    browser.implicitly_wait(10) #페이지 로딩 기다려줌
    browser.maximize_window() # 화면 최대화
    browser.execute_script(f'window.open("{url}","_blank");')
    browser.implicitly_wait(10) #페이지 로딩 기다려줌

    browser.switch_to.window(browser.window_handles[-1]) # 새 탭으로 전환
    browser.implicitly_wait(5) #페이지 로딩 기다려줌

    browser.find_element(By.CSS_SELECTOR,'#root > div > div.Layout_wrap__3uDBh > div > div > div.Login_simple_box__2bfAS > button > img').click()
    time.sleep(3)

    #주문 페이지로 이동
    url = 'https://sell.smartstore.naver.com/o/v3/n/sale/delivery#'
    browser.execute_script(f'window.open("{url}","_blank");')
    browser.switch_to.window(browser.window_handles[-1]) # 새 탭으로 전환
    browser.implicitly_wait(5) #페이지 로딩 기다려줌

    # 발주 준비중인 내역 보기
    browser.find_element(By.CSS_SELECTOR,'#__app_root__ > div > div.napy_sub_content > div:nth-child(3) > div > div:nth-child(2) > ul > li:nth-child(2) > div > a.ico_order_confirm._47sX76YFox._3LuPaa0ngI').click()

    # 가장 위에 있는 주문 내역 선택
    browser.find_element(By.CSS_SELECTOR,'#__app_root__ > div > div.napy_sub_content > div:nth-child(5) > div.npay_grid_area > div.grid > div.tui-grid-container > div.tui-grid-content-area.tui-grid-show-lside-area > div.tui-grid-lside-area > div.tui-grid-body-area > div > div.tui-grid-table-container > table > tbody > tr > td:nth-child(2) > div > a').click()
    time.sleep(2)
    browser.switch_to.window(browser.window_handles[-1])

    # 아이디 선택
    element = browser.find_element(By.CSS_SELECTOR,'#__app_root__ > div > div > div:nth-child(2) > div:nth-child(1) > div > div > div:nth-child(3) > table > tbody > tr:nth-child(4) > td')
    name_value = element.text  # 'text' 속성을 사용하여 element의 텍스트를 가져옵니다.
    pyperclip.copy(name_value)

    # 클립보드에서 텍스트를 가져옴
    copied_text = pyperclip.paste()

    # 가져온 텍스트에 '@naver.com'을 붙임
    email = copied_text + '@naver.com'

    # 만들어진 이메일 주소를 클립보드에 복사
    pyperclip.copy(email)
    time.sleep(1)

    #내게 보낸 메일함으로
    url = 'https://mail.naver.com/v2/read/6/59'
    browser.execute_script(f'window.open("{url}","_blank");')
    browser.switch_to.window(browser.window_handles[-1]) # 새 탭으로 전환
    browser.implicitly_wait(5) #페이지 로딩 기다려줌

    #수정 버튼 클릭
    browser.find_element(By.CSS_SELECTOR,'#content > div.mail_toolbar.type_read > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > button > span').click()
    browser.implicitly_wait(5) #페이지 로딩 기다려줌

    #메일 쓰기 버튼 클릭
    browser.find_element(By.CSS_SELECTOR,'#content > div.mail_toolbar.type_write > div:nth-child(1) > div > button.button_write_change').click()

    #받는사람 칸 클릭
    browser.find_element(By.CSS_SELECTOR,'#recipient_input_element').click()

    #받는사람 이메일 입력
    pyautogui.hotkey('ctrl', 'v')  # 'Ctrl+V'를 누름

    #이메일 보내기
    browser.find_element(By.CSS_SELECTOR,'#content > div.mail_toolbar.type_write > div:nth-child(1) > div > button.button_write_task').click()

    #다시 주문 페이지
    url = 'https://sell.smartstore.naver.com/o/v3/n/sale/delivery#'
    browser.execute_script(f'window.open("{url}","_blank");')
    browser.switch_to.window(browser.window_handles[-1]) # 새 탭으로 전환
    browser.implicitly_wait(5) #페이지 로딩 기다려줌

    # 발주 준비중인 내역 보기
    browser.find_element(By.CSS_SELECTOR,'#__app_root__ > div > div.napy_sub_content > div:nth-child(3) > div > div:nth-child(2) > ul > li:nth-child(2) > div > a.ico_order_confirm._47sX76YFox._3LuPaa0ngI').click()

    # 배송 완료 클릭
    browser.find_element(By.CSS_SELECTOR,'#__app_root__ > div > div.napy_sub_content > div:nth-child(5) > div.npay_grid_area > div.grid > div.tui-grid-container > div.tui-grid-content-area.tui-grid-show-lside-area > div.tui-grid-lside-area > div.tui-grid-body-area > div > div.tui-grid-table-container > table > tbody > tr > td:nth-child(3) > div').click()

get_new_order_list()



발급된 토큰 :  3rrrAD6RF282CvyFkLMXFP
주문 내역 없음
주문 내역 없음


False